In [1]:
# conda deactivate
# conda remove -n finance --all -y

# conda create -n finance python=3.10 -y
# conda activate finance

# pip install pandas yfinance numpy


In [12]:
"""
Download US stock data for:
TSLA, AMZN, NIO, MSFT, AAPL, GOOG, NFLX, COIN

Default period: 2022-01-01 to 2023-12-31
Data source: Yahoo Finance via yfinance
Outputs:
  - data/stock_prices/all_stocks_wide.csv
  - data/stock_prices/{TICKER}.csv  (one file per ticker)
"""

import os
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf

# -------------------------------------------------------------------
# Configuration: you can change these if you want a different period
# -------------------------------------------------------------------
TICKERS = ["TSLA", "AMZN", "NIO", "MSFT", "AAPL", "GOOG", "NFLX", "COIN"]

START_DATE = "2022-01-01"
END_DATE = "2023-12-31"

OUT_DIR = "data/stock_prices"


def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

In [ ]:

def download_prices(tickers, start, end):
    """
    Download daily OHLCV data for a list of tickers.
    Returns a multiindex DataFrame with columns like:
    (Adj Close, TSLA), (Volume, TSLA), ...
    """
    df = yf.download(
        tickers=" ".join(tickers),
        start=start,
        end=end,
        auto_adjust=False,      # keep both Close and Adj Close
        progress=True,
        group_by="ticker"       # easier to split per ticker
    )
    return df


def tidy_single_ticker(df_raw, ticker):
    """
    Convert the raw yahoo format for a single ticker to a flat DataFrame:
    columns: [Date, Open, High, Low, Close, Adj Close, Volume, Ticker]
    """
    # For a single ticker, df_raw[ticker] has OHLCV columns
    tdf = df_raw[ticker].copy()
    tdf = tdf.reset_index()
    tdf["Ticker"] = ticker

    # Standard column ordering if present
    col_order = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "Ticker",]
    tdf = tdf[[c for c in col_order if c in tdf.columns]]
    return tdf


In [ ]:
def main():
    ensure_dir(OUT_DIR)

    print(f"Downloading data for {len(TICKERS)} tickers "
          f"from {START_DATE} to {END_DATE}...")
tesla_data = yf.download("TSLA", start="2025-01-01", end="2025-10-31")
tesla_data.head()
    df_raw = download_prices(TICKERS, START_DATE, END_DATE)

    # Handle case where Yahoo returns a single level columns
    # (happens when only one ticker is requested)
    if not isinstance(df_raw.columns, pd.MultiIndex):
        # single ticker case, wrap to multiindex structure
        ticker = TICKERS[0]
        df_raw = pd.concat({ticker: df_raw}, axis=1)

    all_list = []

    for ticker in TICKERS:
        if ticker not in df_raw.columns.levels[0]:
            print(f"Warning: {ticker} not found in downloaded data. Skipping.")
            continue

        tdf = tidy_single_ticker(df_raw, ticker)

        # Drop days where everything is NaN (market holidays, missing)
        price_cols = [c for c in ["Open", "High", "Low", "Close", "Adj Close", "Volume"] if c in tdf.columns]
        tdf = tdf.dropna(subset=price_cols, how="all")

        # Save per ticker
        out_path = os.path.join(OUT_DIR, f"{ticker}.csv")
        tdf.to_csv(out_path, index=False)
        print(f"Saved {ticker} data to {out_path} "
              f"({len(tdf)} rows).")

        all_list.append(tdf)

    if not all_list:
        print("No data downloaded. Check tickers or date range.")
        return

    # Concatenate all tickers into one long table
    all_df = pd.concat(all_list, ignore_index=True)
    all_df = all_df.sort_values(["Date", "Ticker"])

    all_path = os.path.join(OUT_DIR, "all_stocks_long.csv")
    all_df.to_csv(all_path, index=False)
    print(f"Saved combined dataset to {all_path} "
          f"({len(all_df)} rows).")

    print("Done.")


if __name__ == "__main__":
    main()

[************          25%                       ]  2 of 8 completed

[*********************100%***********************]  8 of 8 completed


Saved TSLA data to data/stock_prices/TSLA.csv (501 rows).
Saved AMZN data to data/stock_prices/AMZN.csv (501 rows).
Saved NIO data to data/stock_prices/NIO.csv (501 rows).
Saved MSFT data to data/stock_prices/MSFT.csv (501 rows).
Saved AAPL data to data/stock_prices/AAPL.csv (501 rows).
Saved GOOG data to data/stock_prices/GOOG.csv (501 rows).
Saved NFLX data to data/stock_prices/NFLX.csv (501 rows).
Saved COIN data to data/stock_prices/COIN.csv (501 rows).
Saved combined dataset to data/stock_prices/all_stocks_long.csv (4008 rows).
Done.


In [ ]:
# TICKERS = ["TSLA", "AMZN", "NIO", "MSFT", "AAPL", "GOOG", "NFLX", "COIN"]
# col_order = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume", "Ticker",]

# So the DataFrame has these effective columns:
#     Date: THe day the stock was traded
#     Open: price at market open
#     High: highest price that day
#     Low: lowest price that day
#     Close: closing price for Company
#     Volume: number of shares traded

tesla_data = yf.download("TSLA", start="2025-01-01", end="2025-10-31")
tesla_data.head()

# tesla_data = panda.read_csv("data/stock_prices/TSLA.csv")
# tesla.head()

/tmp/ipykernel_161682/617805218.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  tesla_data = yf.download("TSLA", start="2025-01-01", end="2025-10-31")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA
Date,,,,,
2025-01-02,379.279999,392.730011,373.040009,390.100006,109710700
2025-01-03,410.440002,411.880005,379.450012,381.480011,95423300
2025-01-06,411.049988,426.429993,401.700012,423.200012,85516500
2025-01-07,394.359985,414.329987,390.000000,405.829987,75699500
2025-01-08,394.940002,402.500000,387.399994,392.950012,73038800


In [9]:
tesla_data = pd.read_csv("data/stock_prices/TSLA.csv")
tesla_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2022-01-03,382.583344,400.356659,378.679993,399.926666,399.926666,103931400,TSLA
1,2022-01-04,396.516663,402.666656,374.350006,383.196655,383.196655,100248300,TSLA
2,2022-01-05,382.216675,390.113342,360.336670,362.706665,362.706665,80119800,TSLA
3,2022-01-06,359.000000,362.666656,340.166656,354.899994,354.899994,90336600,TSLA
4,2022-01-07,360.123322,360.309998,336.666656,342.320007,342.320007,84164700,TSLA


OPTION A. A full Python script to compute all FINCON metrics from one CSV

In [17]:
# This script works for a single stock at a time.
# Save as: compute_fincon_metrics.py

def compute_log_returns(df):
    df = df.sort_values("Date")
    df["LogReturn"] = np.log(df["Adj Close"] / df["Adj Close"].shift(1))
    return df

def compute_pnl(df, actions):
    """
    actions is a sequence of integers the same length as df.
    1 means buy, minus 1 means sell, 0 means hold.
    """
    df["Action"] = actions
    df["PnL"] = df["LogReturn"] * df["Action"]
    return df

def compute_cvar(df, alpha=0.01):
    var_threshold = df["PnL"].quantile(alpha)
    cvar = df[df["PnL"] <= var_threshold]["PnL"].mean()
    return cvar

def compute_cumulative_return(df):
    return df["LogReturn"].sum()

def compute_sharpe_ratio(df, risk_free=0.0):
    rp = df["PnL"].mean()
    sigma = df["PnL"].std()
    if sigma == 0:
        return 0.0
    return (rp - risk_free) / sigma

def compute_max_drawdown(df):
    df["Equity"] = df["PnL"].cumsum()
    df["Peak"] = df["Equity"].cummax()
    df["Drawdown"] = (df["Equity"] - df["Peak"]) / df["Peak"]
    return df["Drawdown"].min()

def compute_momentum(df, window=20):
    df[f"Momentum_{window}"] = df["Adj Close"].pct_change(window)
    return df

def compute_fincon_metrics(csv_path, actions=None):
    df = pd.read_csv(csv_path)
    df = compute_log_returns(df)

    if actions is None:
        # hold strategy for demonstration
        actions = [1] * len(df)

    df = compute_pnl(df, actions)
    df = compute_momentum(df, window=20)

    metrics = {
        "CumulativeReturn": compute_cumulative_return(df),
        "SharpeRatio": compute_sharpe_ratio(df),
        "MaxDrawdown": compute_max_drawdown(df),
        "CVaR": compute_cvar(df),
    }

    return df, metrics


if __name__ == "__main__":
    df, metrics = compute_fincon_metrics("data/stock_prices/TSLA.csv")
    print("Metrics:")
    print(metrics)


Metrics:
{'CumulativeReturn': np.float64(-0.47591885374360987), 'SharpeRatio': np.float64(-0.02498139685962395), 'MaxDrawdown': np.float64(-0.0), 'CVaR': np.float64(-0.12139528954992466)}


OPTION B. A FINCON style dataset builder for multiple stocks

In [18]:
# This script:
#     Loads all eight tickers from your CSV directory
#     Computes all FINCON metrics per stock
#     Produces a combined FINCON dataset
#     Prepares ready to use tables for portfolio tasks
#     Computes return correlations, expected return (mu), and covariance (Sigma)


TICKERS = ["TSLA", "AMZN", "NIO", "MSFT", "AAPL", "GOOG", "NFLX", "COIN"]

def load_stock(path):
    df = pd.read_csv(path)
    df = df.sort_values("Date")
    df["LogReturn"] = np.log(df["Adj Close"] / df["Adj Close"].shift(1))
    df["Return"] = df["Adj Close"].pct_change()
    df["Momentum_20"] = df["Adj Close"].pct_change(20)
    return df

def compute_metrics(df):
    df = df.copy()
    # df["Equity"] = df["LogReturn"].cumsum()
    df["Equity"] = (1 + df["Return"]).cumprod()
    df["Peak"] = df["Equity"].cummax()
    df["Drawdown"] = (df["Equity"] - df["Peak"]) / df["Peak"]

    cvar = df["LogReturn"].quantile(0.01)
    tail = df[df["LogReturn"] <= cvar]["LogReturn"].mean()

    metrics = {
        "CR": df["LogReturn"].sum(),
        "SR": df["LogReturn"].mean() / df["LogReturn"].std(),
        "MDD": df["Drawdown"].min(),
        "CVaR": tail
    }
    return metrics

def build_all_stocks_dataset(data_dir="data"):
    combined = []
    metrics_table = []

    for ticker in TICKERS:
        path = os.path.join(data_dir, f"{ticker}.csv")
        if not os.path.exists(path):
            print(f"Missing CSV for {ticker}, skipping.")
            continue

        df = load_stock(path)
        df["Ticker"] = ticker

        m = compute_metrics(df)
        m["Ticker"] = ticker
        metrics_table.append(m)

        combined.append(df)

    combined_df = pd.concat(combined, ignore_index=True)
    metrics_df = pd.DataFrame(metrics_table)

    return combined_df, metrics_df

def compute_portfolio_statistics(combined_df):
    wide = combined_df.pivot(index="Date", columns="Ticker", values="LogReturn")
    mu = wide.mean()
    Sigma = wide.cov()
    corr = wide.corr()
    return mu, Sigma, corr


if __name__ == "__main__":
    combined_df, metrics_df = build_all_stocks_dataset("data/stock_prices/")
    mu, Sigma, corr = compute_portfolio_statistics(combined_df)

    print("Per stock metrics:")
    print(metrics_df)

    print("Expected returns mu:")
    print(mu)

    print("Covariance matrix Sigma:")
    print(Sigma)

    print("Correlation matrix:")
    print(corr)


Per stock metrics:
         CR        SR       MDD      CVaR Ticker
0 -0.475919 -0.024981 -0.717899 -0.121395   TSLA
1 -0.114689 -0.008551 -0.516759 -0.094740   AMZN
2 -1.305677 -0.051413 -0.774306 -0.157141    NIO
3  0.134466  0.013909 -0.344549 -0.056244   MSFT
4  0.067551  0.007385 -0.300247 -0.055895   AAPL
5 -0.028984 -0.002623 -0.436018 -0.079962   GOOG
6 -0.204519 -0.010964 -0.718566 -0.187485   NFLX
7 -0.367057 -0.011721 -0.869958 -0.212792   COIN
Expected returns mu:
Ticker
AAPL    0.000135
AMZN   -0.000229
COIN   -0.000734
GOOG   -0.000058
MSFT    0.000269
NFLX   -0.000409
NIO    -0.002611
TSLA   -0.000952
dtype: float64
Covariance matrix Sigma:
Ticker      AAPL      AMZN      COIN      GOOG      MSFT      NFLX       NIO  \
Ticker                                                                         
AAPL    0.000335  0.000306  0.000582  0.000283  0.000262  0.000315  0.000402   
AMZN    0.000306  0.000720  0.000927  0.000408  0.000359  0.000510  0.000590   
COIN    0.000582